In [1]:
''' compare RTOFS against alongtrack SD absolute current '''
import numpy as np
import netCDF4 as nc
import matplotlib
import matplotlib.pyplot as plt
import datetime
import os
import sys
from scipy import stats

In [2]:
''' SD data info '''
year = '2023'
platfs_num = ['1031','1040','1042','1045','1057','1064','1065','1068','1083']#['1036','1041','1069']
path_SD = os.path.expanduser('~/Documents/projects/sd-ni-wp/data_manipulate/data_merge_adcp/')
filenames_SD_all = np.sort( os.listdir(path_SD) )
filenames_SD = np.array([])
for file in filenames_SD_all:
    if ('.nc' in file) & (year in file) & (file[-7:-3] in platfs_num):
        filenames_SD = np.append(filenames_SD, file)
print(len(filenames_SD), filenames_SD)
''' model data info '''
data_type = 'surface.now.'
path_model = '/Volumes/disk3/projects/tc2022/tc2023/RTOFS/study_area/'
filenames_model_all = np.sort( os.listdir(path_model) )
filenames_model = np.array([])
for file in filenames_model_all:
    if ('.nc' in file) & (file[:12] == data_type):
        filenames_model = np.append(filenames_model, file)
print(len(filenames_model), filenames_model)

9 ['adcp-raw-merge-2023-SD1031.nc' 'adcp-raw-merge-2023-SD1040.nc'
 'adcp-raw-merge-2023-SD1042.nc' 'adcp-raw-merge-2023-SD1045.nc'
 'adcp-raw-merge-2023-SD1057.nc' 'adcp-raw-merge-2023-SD1064.nc'
 'adcp-raw-merge-2023-SD1065.nc' 'adcp-raw-merge-2023-SD1068.nc'
 'adcp-raw-merge-2023-SD1083.nc']
120 ['surface.now.20230720.nc' 'surface.now.20230721.nc'
 'surface.now.20230722.nc' 'surface.now.20230723.nc'
 'surface.now.20230724.nc' 'surface.now.20230725.nc'
 'surface.now.20230726.nc' 'surface.now.20230727.nc'
 'surface.now.20230728.nc' 'surface.now.20230729.nc'
 'surface.now.20230730.nc' 'surface.now.20230731.nc'
 'surface.now.20230801.nc' 'surface.now.20230802.nc'
 'surface.now.20230803.nc' 'surface.now.20230804.nc'
 'surface.now.20230805.nc' 'surface.now.20230806.nc'
 'surface.now.20230807.nc' 'surface.now.20230808.nc'
 'surface.now.20230809.nc' 'surface.now.20230810.nc'
 'surface.now.20230811.nc' 'surface.now.20230812.nc'
 'surface.now.20230813.nc' 'surface.now.20230814.nc'
 'surface.n

In [3]:
''' load all SD platf data & append specific variables to the dictionary '''
SDs_data = {}
vars_SD_want = ['time','longitude','latitude','vel_east','vel_north']
# print(SDs_data)
for i, file in enumerate( filenames_SD ):
    platf_num = file[-7:-3]
    print('start reading SD',platf_num,'nc data')
    ds = nc.Dataset(path_SD+file)
    # print(ds)
    ''' read variables '''
    for j, var_str in enumerate( vars_SD_want ):
        str_exec = var_str+' = ds.variables["'+var_str+'"][:]'
        exec(str_exec)
        if 'vel_' in var_str: ### record the shallowest depth
            str_exec = var_str+' = '+var_str + '[:,0]'
            # print(str_exec)
            exec(str_exec)
        if 'time' in var_str:
            time = np.array([(datetime.datetime(int(year),1,1)+datetime.timedelta(seconds=item)).timestamp() for item in time])
        # print(var_str,eval(var_str).shape)
        ''' write data to dictionary '''
        SDs_data[platf_num+'-'+var_str] = eval(var_str)
    ds.close()

start reading SD 1031 nc data
start reading SD 1040 nc data
start reading SD 1042 nc data
start reading SD 1045 nc data
start reading SD 1057 nc data
start reading SD 1064 nc data
start reading SD 1065 nc data
start reading SD 1068 nc data
start reading SD 1083 nc data


In [4]:
''' loop through all the model surface.now daily data and find the collocated values to SD alongtrack data '''
# vars_str_check = ['LONGITUDE','LATITUDE','MT']
model_match_SDs_data = {}
vars_str = ['U_VELOCITY','V_VELOCITY']
dt_search = datetime.timedelta(minutes=2.5).total_seconds()
dx_search = 0.1
for i, file in enumerate( filenames_model ):
    ds = nc.Dataset(path_model+file)
    print('Reading model time/lon/lat:',file)
    # print(ds)
    ''' read model data '''
    if 'u_model' in locals():
        del u_model, v_model
    lon_model = ds.variables['LONGITUDE'][:][0,:]
    lon_model[lon_model> 180] = lon_model[lon_model> 180]-360
    lat_model = ds.variables['LATITUDE'][:][:,0]
    timestamp_model = np.array([(datetime.datetime(1900,12,31)+datetime.timedelta(days=item)).timestamp() for item in ds.variables['MT'][:]])
                    
    ''' look at one SD track at a time '''
    for s, platf_num in enumerate(platfs_num):
        ''' read SD data '''
        print('Reading SD',platf_num)
        timestamp_SD = SDs_data[platf_num+'-time']
        lon_SD = SDs_data[platf_num+'-longitude']
        lat_SD = SDs_data[platf_num+'-latitude']
        u_SD = SDs_data[platf_num+'-vel_east']
        v_SD = SDs_data[platf_num+'-vel_north']
        for t in range( len(timestamp_SD) ): ### loop through the SD track
            # print(datetime.datetime.fromtimestamp(timestamp_SD[t]))
            dt_sec = np.abs( timestamp_model-timestamp_SD[t])
            if any(dt_sec< dt_search):
                it_model = np.argmin(dt_sec)
                ilon_model = np.argmin( np.abs(lon_model-lon_SD[t]) )
                ilat_model = np.argmin( np.abs(lat_model-lat_SD[t]) )
                if (np.abs(lon_model[ilon_model]-lon_SD[t])< dx_search) & \
                    (np.abs(lat_model[ilat_model]-lat_SD[t])< dx_search):
                    if 'u_model' not in locals():
                        print('Reading model u/v:',file)
                        u_model = ds.variables['U_VELOCITY'][:]
                        v_model = ds.variables['V_VELOCITY'][:]
                    # print(platf_num,datetime.datetime.fromtimestamp(timestamp_SD[t]), lon_model[ilon_model],lon_SD[t],lat_model[ilat_model],lat_SD[t])
                    ''' append the collocated values in the dictionary '''
                    if platf_num+'-time' not in model_match_SDs_data:
                        model_match_SDs_data[platf_num+'-time'] = timestamp_SD[t]
                        model_match_SDs_data[platf_num+'-longitude'] = lon_SD[t]
                        model_match_SDs_data[platf_num+'-latitude'] = lat_SD[t]
                        model_match_SDs_data[platf_num+'-u'] = np.array([u_SD[t], u_model[it_model,0,ilat_model,ilon_model]])
                        model_match_SDs_data[platf_num+'-v'] = np.array([v_SD[t], v_model[it_model,0,ilat_model,ilon_model]])
                    else:
                        model_match_SDs_data[platf_num+'-time'] = np.append(model_match_SDs_data[platf_num+'-time'], timestamp_SD[t])
                        model_match_SDs_data[platf_num+'-longitude'] = np.append(model_match_SDs_data[platf_num+'-longitude'],lon_SD[t])
                        model_match_SDs_data[platf_num+'-latitude'] = np.append(model_match_SDs_data[platf_num+'-latitude'],lat_SD[t])
                        data_app = np.array([u_SD[t], u_model[it_model,0,ilat_model,ilon_model]])
                        model_match_SDs_data[platf_num+'-u'] = np.column_stack( (model_match_SDs_data[platf_num+'-u'], data_app) )
                        data_app = np.array([v_SD[t], v_model[it_model,0,ilat_model,ilon_model]])
                        model_match_SDs_data[platf_num+'-v'] = np.column_stack( (model_match_SDs_data[platf_num+'-v'], data_app) )
    ds.close()
    # break

Reading model time/lon/lat: surface.now.20230720.nc
Reading SD 1031
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading model u/v: surface.now.20230720.nc
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.20230721.nc
Reading SD 1031
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading model u/v: surface.now.20230721.nc
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.20230722.nc
Reading SD 1031
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading model u/v: surface.now.20230722.nc
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.20230723.nc
Reading SD 1031
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading model u/v: surface.now.20230723.nc
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.2023

/var/folders/kq/vqqqnf_x6vd47tpg4lf57cjr0002p5/T/ipykernel_41627/2062948359.py:47: UserWarning: Warning: converting a masked element to nan.
  model_match_SDs_data[platf_num+'-u'] = np.array([u_SD[t], u_model[it_model,0,ilat_model,ilon_model]])
/var/folders/kq/vqqqnf_x6vd47tpg4lf57cjr0002p5/T/ipykernel_41627/2062948359.py:48: UserWarning: Warning: converting a masked element to nan.
  model_match_SDs_data[platf_num+'-v'] = np.array([v_SD[t], v_model[it_model,0,ilat_model,ilon_model]])
/var/folders/kq/vqqqnf_x6vd47tpg4lf57cjr0002p5/T/ipykernel_41627/2062948359.py:53: UserWarning: Warning: converting a masked element to nan.
  data_app = np.array([u_SD[t], u_model[it_model,0,ilat_model,ilon_model]])
/var/folders/kq/vqqqnf_x6vd47tpg4lf57cjr0002p5/T/ipykernel_41627/2062948359.py:55: UserWarning: Warning: converting a masked element to nan.
  data_app = np.array([v_SD[t], v_model[it_model,0,ilat_model,ilon_model]])


Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.20230813.nc
Reading SD 1031
Reading model u/v: surface.now.20230813.nc
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.20230814.nc
Reading SD 1031
Reading model u/v: surface.now.20230814.nc
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.20230815.nc
Reading SD 1031
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading model u/v: surface.now.20230815.nc
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1083
Reading model time/lon/lat: surface.now.20230816.nc
Reading SD 1031
Reading SD 1040
Reading SD 1042
Reading SD 1045
Reading model u/v: surface.now.20230816.nc
Reading SD 1057
Reading SD 1064
Reading SD 1065
Reading SD 1068
Reading SD 1

In [5]:
# plt.pcolormesh(lon_model)
# plt.colorbar()
# plt.plot(np.array([datetime.datetime.fromtimestamp(item) for item in SDs_data['1036-time']]),'.')
# dt_search = datetime.timedelta(minutes=2.5).total_seconds()
# print(dt_search)
# any(dt_sec> 3.28e6)
# plt.plot(lon_model[0,:])
# u_model = ds.variables['U_VELOCITY'][:]
# v_model = ds.variables['V_VELOCITY'][:]
# print(lon_model.shape, lat_model.shape, u_model.shape)
# print(u_model[it_model,0,ilat_model,ilon_model])
# for key in model_match_SDs_data.keys():
#     print(key, model_match_SDs_data[key].shape)
''' remove NaN values '''
for i, platf_num in enumerate( platfs_num ):
    isNaN = np.where( np.isnan(model_match_SDs_data[platf_num+'-u'][0,:]) | \
                     np.isnan(model_match_SDs_data[platf_num+'-u'][1,:]) )[0]
    print(platf_num, isNaN.shape, model_match_SDs_data[platf_num+'-u'].shape)
    if len(isNaN) > 0:
        model_match_SDs_data[platf_num+'-longitude'] = np.delete(model_match_SDs_data[platf_num+'-longitude'], isNaN)
        model_match_SDs_data[platf_num+'-latitude'] = np.delete(model_match_SDs_data[platf_num+'-latitude'], isNaN)
        model_match_SDs_data[platf_num+'-time'] = np.delete(model_match_SDs_data[platf_num+'-time'], isNaN)
        model_match_SDs_data[platf_num+'-u'] = np.delete(model_match_SDs_data[platf_num+'-u'], isNaN, axis=1 )
        model_match_SDs_data[platf_num+'-v'] = np.delete(model_match_SDs_data[platf_num+'-v'], isNaN, axis=1 )
        print(platf_num, model_match_SDs_data[platf_num+'-u'].shape)
    

1031 (0,) (2, 1719)
1040 (0,) (2, 883)
1042 (0,) (2, 672)
1045 (3,) (2, 2292)
1045 (2, 2289)
1057 (0,) (2, 1257)
1064 (3,) (2, 437)
1064 (2, 434)
1065 (0,) (2, 917)
1068 (0,) (2, 1476)
1083 (0,) (2, 2138)


In [6]:
# ''' plot 2D-histogram of u & v '''
# nrow = 2; ncol = 4
# bins = np.arange(-1.25,1.25,0.05)
# xlim = [-1.2,1.2]
# x0 = np.arange(bins[0],bins[-1]+0.1,0.1)
# histlim = [0,0.02]
# ''' '''
# plt.clf()
# plt.rcParams.update({'font.size': 13})
# fig, ax = plt.subplots(nrows=nrow,ncols=ncol)
# fig.set_size_inches(20, 9)
# ''' plot individual SD '''
# for i, platf_num in enumerate(platfs_num):
#     print('Shape of',platf_num,'u/v:',model_match_SDs_data[platf_num+'-u'].shape)
#     ''' u '''
#     plt.subplot(nrow,ncol,i+1)
#     x = model_match_SDs_data[platf_num+'-u'][0,:]
#     y = model_match_SDs_data[platf_num+'-u'][1,:]
#     slope, intercept, r, p, std_err = stats.linregress(x,y)
#     h, xedges,yedges = np.histogram2d(x, y, bins=bins,density=True)
#     cs = plt.pcolormesh(xedges, yedges, h/h.sum(), cmap='turbo',vmin=histlim[0],vmax=histlim[1])
#     plt.plot( x0, x0*slope+intercept, 'm--')
#     plt.xlabel('U$_{SD'+platf_num+'}$ (m s$^{-1}$)')
#     plt.ylabel('U$_{RTOFS}$ (m s$^{-1}$)')
#     plt.title('y={:.2f}x+{:.2E}. r= {:.2f},\np= {:.2f}, std_err= {:.3f}'.format(slope,intercept,r,p,std_err), fontsize=12)
#     ''' v '''
#     plt.subplot(nrow,ncol,i+ncol+1)
#     x = model_match_SDs_data[platf_num+'-v'][0,:]
#     y = model_match_SDs_data[platf_num+'-v'][1,:]
#     slope, intercept, r, p, std_err = stats.linregress(x,y)
#     h, xedges,yedges = np.histogram2d(x, y, bins=bins,density=True)
#     cs = plt.pcolormesh(xedges, yedges, h/h.sum(), cmap='turbo',vmin=histlim[0],vmax=histlim[1])
#     plt.plot( x0, x0*slope+intercept, 'm--')
#     plt.xlabel('V$_{SD'+platf_num+'}$ (m s$^{-1}$)')
#     plt.ylabel('V$_{RTOFS}$ (m s$^{-1}$)')
#     plt.title('y={:.2f}x+{:.2E}. r= {:.2f},\np= {:.2f}, std_err= {:.3f}'.format(slope,intercept,r,p,std_err), fontsize=12)
#     ''' stack data '''
#     if i == 0:
#         u_all = model_match_SDs_data[platf_num+'-u']
#         v_all = model_match_SDs_data[platf_num+'-v']
#     else:
#         u_all = np.column_stack( (u_all, model_match_SDs_data[platf_num+'-u']) )
#         v_all = np.column_stack( (v_all, model_match_SDs_data[platf_num+'-v']) )
# ''' plot all SDs '''
# ''' u '''
# plt.subplot(nrow,ncol,ncol)
# x = u_all[0,:]
# y = u_all[1,:]
# slope, intercept, r, p, std_err = stats.linregress(x,y)
# h, xedges,yedges = np.histogram2d(x, y, bins=bins,density=True)
# cs = plt.pcolormesh(xedges, yedges, h/h.sum(), cmap='turbo',vmin=histlim[0],vmax=histlim[1])
# plt.plot( x0, x0*slope+intercept, 'm--')
# plt.xlabel('U$_{SD}$ (m s$^{-1}$)')
# plt.ylabel('U$_{RTOFS}}$ (m s$^{-1}$)')
# plt.title('y={:.2f}x+{:.2E}. r= {:.2f},\np= {:.2f}, std_err= {:.3f}'.format(slope,intercept,r,p,std_err), fontsize=12)
# ''' v '''
# plt.subplot(nrow,ncol,ncol*2)
# x = v_all[0,:]
# y = v_all[1,:]
# slope, intercept, r, p, std_err = stats.linregress(x,y)
# h, xedges,yedges = np.histogram2d(x, y, bins=bins,density=True)
# cs = plt.pcolormesh(xedges, yedges, h/h.sum(), cmap='turbo',vmin=histlim[0],vmax=histlim[1])
# plt.plot( x0, x0*slope+intercept, 'm--')
# plt.xlabel('V$_{SD}$ (m s$^{-1}$)')
# plt.ylabel('V$_{RTOFS}}$ (m s$^{-1}$)')
# plt.title('y={:.2f}x+{:.2E}. r= {:.2f},\np= {:.2f}, std_err= {:.3f}'.format(slope,intercept,r,p,std_err), fontsize=12)
# ''' add colorbar '''
# axf = fig.add_axes([0.91,0.15,0.005,0.7])
# axf = plt.colorbar(cs,orientation='vertical',cax=axf,extend='both')
# # axf.set_label('',fontsize=14)
# ''' settings '''
# for i in range(nrow*ncol):
#     plt.subplot(nrow,ncol,i+1)
#     # plt.axis('equal')
#     plt.plot(xlim,xlim,'--',color='lightgray')
#     plt.grid()
#     plt.xticks(np.arange(xlim[0],xlim[1],0.4))
#     plt.yticks(np.arange(xlim[0],xlim[1],0.4))
#     plt.xlim(xlim)
#     plt.ylim(xlim)
# ''' '''
# plt.suptitle('2D histogram of u & v between SD$_{6m}$ ('+','.join(platfs_num)+') & RTOFS')
# plt.subplots_adjust(left=0.1,bottom=0.1,right=0.9,top=0.9,wspace=0.3,hspace=0.35)
# ''' save figure '''
# plt.savefig('2Dhist_uv_SD(6m)('+year+':'+'-'.join(platfs_num)+')_RTOFS.png',dpi=400,bbox_inches='tight')

In [ ]:
''' write data out to text files - by drone '''
for i, platf_num in enumerate( platfs_num ):
    vars_str_in_dict = ['time','longitude','latitude','u','v']
    vars_str_out = ['timestamp','longitude','latitude','u(SD 6m) cur (m/s)','u(RTOFS) cur (m/s)','v(SD 6m) cur (m/s)','u(RTOFS) cur (m/s)']
    if 'out_array' in locals():
        del out_array
    ''' column stack the values to a 2D array '''
    for j, var_str_in_dict in enumerate( vars_str_in_dict):
        data = model_match_SDs_data[platf_num+'-'+var_str_in_dict]
        if data.ndim == 1:
            if 'out_array' not in locals():
                out_array = data
            else:
                out_array = np.column_stack( (out_array, data) )
        if data.ndim == 2:
            for k in range( data.shape[0] ):
                out_array = np.column_stack( (out_array, data[k,:]))
    print(platf_num, out_array.shape)
    ''' write 2D array to text file '''
    fn_out = 'timeseries_cur_match_'+year+'-SD'+platf_num+'_RTOFS.txt'
    np.savetxt(fn_out, out_array, delimiter=' ', header=' '.join(vars_str_out))

1031 (1719, 7)
1040 (883, 7)
1042 (672, 7)
1045 (2289, 7)
1057 (1257, 7)
1064 (434, 7)
1065 (917, 7)
1068 (1476, 7)
1083 (2138, 7)
